# Imports

In [16]:
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup


# Test

In [10]:

# Create Chrome options if needed (optional)
opts = Options()
opts.add_argument("--headless")  # Example option, run Chrome in headless mode

# Initialize the WebDriver using webdriver-manager
web_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# Example usage
web_driver.get("https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24")
print(web_driver.title)  # Prints the title of the page



Season Leaders | Stats | NBA.com


In [15]:
# Optionally, save the HTML to a file
with open('page_source.html', 'w', encoding='utf-8') as file:
    file.write(web_driver.page_source)

In [19]:
# Assuming you've saved the HTML content in 'page_source.html'
with open('page_source.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(html_content, 'html.parser')

for bracket in soup:
    print(bracket)

<html data-build="15045" data-theme="" data-version="4.35.0" lang="en"><head><meta charset="utf-8"/><link href="/site-manifest.json" rel="manifest"/><meta content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/><script src="/newrelic/newrelic-prod.js"></script><script id="theme-loader" src="/theme-loader.js"></script><title>Season Leaders | Stats | NBA.com</title><meta content="The official leaders for the latest NBA season. Features the ability to filter by season, season type, per mode, stat category, and more." name="description"/><link href="/favicon.ico" rel="icon"/><link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/apple-touch-icon.png" rel="apple-touch-icon"/><link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="120x120"/><link hr

# Constants

In [ ]:
TABLE_NAME = "player_stats"

# Functions

In [7]:
def extract_nba_stats(html):
  pass

In [ ]:
years = [str(year) + "-" +str(year+1)[2:] for year in range(1951, 2024)]
print(years)

['1951-52', '1952-53', '1953-54', '1954-55', '1955-56', '1956-57', '1957-58', '1958-59', '1959-60', '1960-61', '1961-62', '1962-63', '1963-64', '1964-65', '1965-66', '1966-67', '1967-68', '1968-69', '1969-70', '1970-71', '1971-72', '1972-73', '1973-74', '1974-75', '1975-76', '1976-77', '1977-78', '1978-79', '1979-80', '1980-81', '1981-82', '1982-83', '1983-84', '1984-85', '1985-86', '1986-87', '1987-88', '1988-89', '1989-90', '1990-91', '1991-92', '1992-93', '1993-94', '1994-95', '1995-96', '1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']


In [ ]:
for year in years:
  base_url = f"https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season={year}"
  response = requests.get(base_url)